In [1]:
# Necessary packages
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import os
import cv2
import time
import pickle
import random
from tqdm import tqdm
from astropy.io import fits
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv1D, MaxPooling1D

In [2]:
# CNN Model
X = pickle.load(open("X.pickle", "rb"))
Y = pickle.load(open("Y.pickle", "rb"))

conv_layers = [3]
layer_sizes = [16]

for conv_layer in conv_layers:
    for layer_size in layer_sizes:
        NAME = "{}-conv-{}-nodes-0.15noise-{}".format(conv_layer, layer_size, int(time.time()))
        print(NAME)

        model = Sequential()

        for l in range(conv_layer):
            model.add(Conv1D(kernel_size=layer_size,
                             filters=32,
                             input_shape=X.shape[1:],
                             activation='relu'))
            model.add(Conv1D(kernel_size=layer_size,
                             filters=16,
                             activation='relu'))
            model.add(MaxPooling1D(pool_size=4))

        model.add(Flatten())
        model.add(Dense(3))
        model.add(Activation('softmax'))

        tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))

        model.compile(loss='sparse_categorical_crossentropy',
                      optimizer='adam',
                      metrics=['accuracy'])

        model.fit(X, Y,
                  batch_size=25,
                  epochs=15,
                  validation_split=0.2,
                  callbacks=[tensorboard])

3-conv-16-nodes-0.15noise-1559674545
Instructions for updating:
Colocations handled automatically by placer.
Train on 10800 samples, validate on 2700 samples
Epoch 1/15
10800/10800 [==============================] - 63s 6ms/sample - loss: 1.0427 - acc: 0.4123 - val_loss: 0.8452 - val_acc: 0.5830
Epoch 2/15
10800/10800 [==============================] - 62s 6ms/sample - loss: 0.8159 - acc: 0.6069 - val_loss: 0.7795 - val_acc: 0.6278
Epoch 3/15
10800/10800 [==============================] - 63s 6ms/sample - loss: 0.7669 - acc: 0.6315 - val_loss: 0.7624 - val_acc: 0.6211
Epoch 4/15
10800/10800 [==============================] - 63s 6ms/sample - loss: 0.7421 - acc: 0.6450 - val_loss: 0.7233 - val_acc: 0.6496
Epoch 5/15
10800/10800 [==============================] - 63s 6ms/sample - loss: 0.7257 - acc: 0.6535 - val_loss: 0.7984 - val_acc: 0.6178
Epoch 6/15
10800/10800 [==============================] - 62s 6ms/sample - loss: 0.7208 - acc: 0.6603 - val_loss: 0.7291 - val_acc: 0.6563
Epoch 7/

In [3]:
# Save the best model for predictions
model.save(NAME)